# Predicting Demonstration Size on Demographic Characteristics and Salience

## GW DATS 6103: Introduction to Data Mining Final Project
### Name: Alexander D. Silberman

Note: Data is limited to 2017–2019, as COVID may substantially throw off the data.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from census import Census
from us import states

import censusdis.data as ced
from censusdis.datasets import ACS1_PROFILE
from censusdis import states

## Data Acquisition and Wrangling

### Results Dataset: Crowd Counting Consortium Phase 1 (2017-2020)

Our results dataset: Crowd Counting Consortium Phase 1 (2017-2020): https://ash.harvard.edu/programs/crowd-counting-consortium/

In [86]:
crowd_data = pd.read_csv("C:/Users/alexa/Code/GW DATS/6103 12 Intro to Data Mining/Final Project/data/crowd_counting_consortium_2017-2020/ccc_compiled_20172020.csv",encoding_errors="replace")
crowd_data.head(5)

C:\Users\alexa\AppData\Local\Temp\ipykernel_15548\206043875.py:1: DtypeWarning: Columns (6,22,31,32,33,34,35,36,37,38,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  crowd_data = pd.read_csv("C:/Users/alexa/Code/GW DATS/6103 12 Intro to Data Mining/Final Project/data/crowd_counting_consortium_2017-2020/ccc_compiled_20172020.csv",encoding_errors="replace")


,date,locality,state,location_detail,online,type,macroevent,actors,claims,valence,...,source_28,source_29,source_30,notes,lat,lon,resolved_locality,resolved_county,resolved_state,fips_code
0,2017-01-01,Washington,DC,Lafayette Square Park,0.0,vigil,NaN,NaN,"for banning nuclear weapons, for peace",0.0,...,NaN,NaN,NaN,White House Peace Vigil continuous since June ...,38.907192,-77.036871,Washington,District of Columbia,DC,11001.0
1,2017-01-01,Mankato,MN,NaN,0.0,vigil,NaN,Peace Vigil Mankato,for peace,0.0,...,NaN,NaN,NaN,every Sunday since 2001,44.163578,-93.999400,Mankato,Blue Earth County,MN,27013.0
2,2017-01-01,Minneapolis,MN,U.S. Bank Stadium,0.0,protest; banner drop,NaN,general protestors,"against the Dakota Access Pipeline, for indige...",1.0,...,NaN,NaN,NaN,hung banner from stadium roof during NFL game,44.977753,-93.265011,Minneapolis,Hennepin County,MN,27053.0
3,2017-01-01,Little Compton,RI,Town Green,0.0,vigil,NaN,Sakonnet Peace Alliance,"for peace, for gun control, for climate action",1.0,...,NaN,NaN,NaN,every Sunday since 2003,41.510103,-71.171156,Little Compton,Newport County,RI,44005.0
4,2017-01-01,Oak Ridge,TN,Y-12 National Security Complex,0.0,vigil,NaN,Oak Ridge Environmental Peace Alliance,for abolishing nuclear weapons,0.0,...,NaN,NaN,NaN,every Sunday since the late 1990s,36.010356,-84.269645,Oak Ridge,Anderson County,TN,47001.0


In [87]:
# source_cols = np.array(["source_"]*30) + np.arange(1,31).astype(str)
# crowd_data.drop(columns=source_cols, inplace=True)
# crowd_data.head(5)


### American Community Survey

In [173]:
api_key = "29e7dfea2f8b253a0a10ccd9626f78e49f4f0a4f"

# census_vars = {
#     "NAME":"name",
#     "DP05_0001E":"total_population",
# }

census_vars = ced.variables.search(ACS1_PROFILE, 2017)
census_vars = census_vars[census_vars["GROUP"] != "N/A"]
census_vars

census_data = [ced.download(ACS1_PROFILE, year, pd.concat([pd.Series(["NAME"]), census_vars["VARIABLE"]], ignore_index=True), 
                           metropolitan_statistical_area_micropolitan_statistical_area ="*", 
                           api_key=api_key,
                           ).set_index("METROPOLITAN_STATISTICAL_AREA_MICROPOLITAN_STATISTICAL_AREA")# .rename(columns=census_vars)
               for year in range(2017,2020)]


census_data = pd.concat(census_data, keys = np.arange(2017,2020))
census_data

NAME  \
     METROPOLITAN_STATISTICAL_AREA_MICROPOLITAN_STAT...                                                 
2017 44220                                                                 Springfield, OH Metro Area   
     44300                                                               State College, PA Metro Area   
     44340                                                                  Statesboro, GA Micro Area   
     44420                                                         Staunton-Waynesboro, VA Metro Area   
     44620                                                               Stevens Point, WI Micro Area   
...                                                                                               ...   
2019 49620                                                                York-Hanover, PA Metro Area   
     49660                                               Youngstown-Warren-Boardman, OH-PA Metro Area   
     49700                                                                   Yuba City, CA Metro Area   
     49740                                                                        Yuma, AZ Metro Area   
     49780                                                                  Zanesville, OH Micro Area   

                                                         DP02PR_0001E  \
     METROPOLITAN_STATISTICAL_AREA_MICROPOLITAN_STAT...                 
2017 44220                                                        NaN   
     44300                                                        NaN   
     44340                                                        NaN   
     44420                                                        NaN   
     44620                                                        NaN   
...                                                               ...   
2019 49620                                                        NaN   
     49660                                                        NaN   
     49700                                                        NaN   
     49740                                                        NaN   
     49780                                                        NaN   

                                                         DP02PR_0001PE  \
     METROPOLITAN_STATISTICAL_AREA_MICROPOLITAN_STAT...                  
2017 44220                                                         NaN   
     44300                                                         NaN   
     44340                                                         NaN   
     44420                                                         NaN   
     44620                                                         NaN   
...                                                                ...   
2019 49620                                                         NaN   
     49660                                                         NaN   
     49700                                                         NaN   
     49740                                                         NaN   
     49780                                                         NaN   

                                                         DP02PR_0002E  \
     METROPOLITAN_STATISTICAL_AREA_MICROPOLITAN_STAT...                 
2017 44220                                                        NaN   
     44300                                                        NaN   
     44340                                                        NaN   
     44420                                                        NaN   
     44620                                                        NaN   
...                                                               ...   
2019 49620                                                        NaN   
     49660                                                        NaN   
     49700                                                        NaN   
     49740     

### Most Important Problem Dataset, Second Release (2024)

In [69]:
mipd_cols = ["studyid",
             "entry",
             "fw_start",
             "fw_end",
             "sample",
             "oversample",
             "weightavailable",
             "short",
             "id",
             "id_anes",
             "weight",
             "mipcode1_r1",
             "mipsubcode1_r1",
             "mipareacode1_r1",
             "mipcode2_r1",
             "mipsubcode2_r1",
             "mipareacode2_r1",
             "mipcode3_r1",
             "mipsubcode3_r1",
             "mipareacode3_r1",
             "mipcode4_r1",
             "mipsubcode4_r1",
             "mipareacode4_r1",
             "mipcode5_r1",
             "mipsubcode5_r1",
             "mipareacode5_r1",
             "mipcode6_r1",
             "mipsubcode6_r1",
             "mipareacode6_r1",
             "mipcode7_r1",
             "mipsubcode7_r1",
             "mipareacode7_r1",
             "mipcode8_r1",
             "mipsubcode8_r1",
             "mipareacode8_r1",
             "mipcode9_r1",
             "mipsubcode9_r1",
             "mipareacode9_r1",
             "mipcode10_r1",
             "mipsubcode10_r1",
             "mipareacode10_r1"]

issue_data_full = pd.DataFrame(columns=mipd_cols)

with pd.read_stata("C:\\Users\\alexa\\Code\\GW DATS\\6103 12 Intro to Data Mining\\Final Project\data\\2024-MIPD\\USMISC2024-MIPD_ind.dta", 
                   chunksize=10000, columns=mipd_cols, convert_categoricals=False) as issue_reader:
    for chunk in issue_reader:
        issue_data_full = pd.concat([issue_data_full, chunk],ignore_index=True)


C:\Users\alexa\AppData\Local\Temp\ipykernel_16328\1607938453.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  issue_data_full = pd.concat([issue_data_full, chunk],ignore_index=True)
C:\Users\alexa\AppData\Local\Temp\ipykernel_16328\1607938453.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  issue_data_full = pd.concat([issue_data_full, chunk],ignore_index=True)
C:\Users\alexa\AppData\Local\Temp\ipykernel_16328\1607938453.py:48: FutureWarning: The behavior of DataFrame concatenation with 

In [70]:
len(issue_data_full)

1009177

There are 1009177 observations in this dataset. These will be limited to one year prior to a given demonstration. As our earliest data is for 2017, no surveys taking place before 2016-01-01 shall be considered.

In [74]:
issue_data = issue_data_full[issue_data_full["fw_end"] > np.datetime64("2016-01-01")]
del(issue_data_full)
len(issue_data)

10725

With this limitation, there are only 10725 observations to deal with.

In [73]:
issue_data.describe()

,entry,fw_start,fw_end,id_anes,weight,mipcode1_r1,mipsubcode1_r1,mipareacode1_r1,mipcode2_r1,mipsubcode2_r1,...,mipareacode7_r1,mipcode8_r1,mipsubcode8_r1,mipareacode8_r1,mipcode9_r1,mipsubcode9_r1,mipareacode9_r1,mipcode10_r1,mipsubcode10_r1,mipareacode10_r1
count,3504.0,10725,10725,0.0,10725.000000,10688.000000,2.061000e+03,136.000000,6107.000000,1372.000,...,0.0,54.000000,4.400000e+01,0.0,10.0,10.0,0.0,0.0,0.0,0.0
mean,1.0,2017-02-12 20:57:23.916083712,2017-02-17 09:21:54.125874432,NaN,1.290072,719.463323,7.225414e+05,826.264706,540.940560,644364.000,...,NaN,898.037037,1.011001e+06,NaN,402.0,402002.0,NaN,NaN,NaN,NaN
min,1.0,2016-01-07 00:00:00,2016-01-10 00:00:00,NaN,0.059000,101.000000,1.030010e+05,365.000000,102.000000,103001.000,...,NaN,401.000000,1.011001e+06,NaN,402.0,402002.0,NaN,NaN,NaN,NaN
25%,1.0,2016-09-27 00:00:00,2016-09-29 00:00:00,NaN,0.587500,208.000000,5.040010e+05,686.250000,204.000000,403001.000,...,NaN,1011.000000,1.011001e+06,NaN,402.0,402002.0,NaN,NaN,NaN,NaN
50%,1.0,2017-01-04 00:00:00,2017-01-09 00:00:00,NaN,0.956882,602.000000,6.080010e+05,731.000000,506.000000,604005.000,...,NaN,1011.000000,1.011001e+06,NaN,402.0,402002.0,NaN,NaN,NaN,NaN
75%,1.0,2018-01-14 00:00:00,2018-01-18 00:00:00,NaN,1.552900,1012.000000,1.011007e+06,731.000000,901.000000,1010004.000,...,NaN,1011.000000,1.011001e+06,NaN,402.0,402002.0,NaN,NaN,NaN,NaN
max,1.0,2018-06-05 00:00:00,2018-06-12 00:00:00,NaN,7.612903,1504.000000,1.012045e+06,1600.000000,1504.000000,1012045.000,...,NaN,1011.000000,1.011001e+06,NaN,402.0,402002.0,NaN,NaN,NaN,NaN
std,0.0,NaN,NaN,NaN,1.158380,500.999539,3.052793e+05,421.284847,379.682161,312522.625,...,NaN,239.178071,6.322257e-01,NaN,0.0,0.0,NaN,NaN,NaN,NaN
